In [1]:
import pandas as pd
import numpy as np
import re
import string
import random

from pythainlp.util import normalize, isthai
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### TFIDF vector

In [152]:
numberReview = 500

In [153]:
review = pd.read_csv("../data/file/review/review_500.csv")
review.head(5)

,placeName,review
0,เซ็นทรัลเวิลด์,ห้างใหญ่มาก อย่างที่กล่าวขานก็คือเดินจนหลง มาก...
1,เซ็นทรัลเวิลด์,ศูนย์การค้า Central wOrld 🛒🛍️ เป็นศูนย์การค้า ...
2,เซ็นทรัลเวิลด์,ช่วงเทศกาลเที่ยวกันอย่าประมาท หน้ากากอนามัยถอด...
3,เซ็นทรัลเวิลด์,เดินทางง่ายสะดวก ไม่ว่าจะเป็นรถส่วนตัว แท็กซี่...
4,เซ็นทรัลเวิลด์,สนุกกับช่วงเวลาแห่งการเฉลิมฉลองกับครอบครัวและค...


In [154]:
# Nan count data
sum(review['review'].isnull())

1480

In [155]:
review['review'] = review['review'].astype(str)

In [156]:
def isGoogleTranslateAndNaN(text: str):
    # Google Translae and nan value
    if (text.find("(แปลโดย Google)") > -1 or text=='nan'):
        return np.nan
    return text

def filterPlaceFromReviewNumber(review : pd.DataFrame, numberReview = 500):
    placeName = (review['placeName'].value_counts()[review['placeName'].value_counts() == numberReview]).keys()
    
    review['placeName'] = review['placeName'].map(lambda x : x if x in placeName else np.nan)
    review.dropna(inplace=True)
    return review

def removeEmoji(string):
    emoj = re.compile("["
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
                      "]+", re.UNICODE)
    return emoj.sub(r'', string)

def cleanText(text:str):
    """
    Normalize Function
    - Remove zero-width spaces
    - Remove duplicate spaces
    - Reorder tone marks and vowels to standard order/spelling
    - Remove duplicate vowels and signs
    - Remove duplicate tone marks
    - Remove dangling non-base characters at the beginning of text
    """
    text = removeEmoji(text)
    text = normalize(text)

    # remove <>, #, punctutaion in order
    text = re.sub(r'<.*?>','', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'[a-z|A-Z|\d|(\a-z)]', '', text)
    text = re.sub(r'…', '', text)
    # URL pattern
    text = re.sub(r"^[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)$", '', text)
    for c in string.punctuation:
        text = re.sub(r'\{}'.format(c), '', text)

    return text

def text_tokenizer(text:str):
    return [term for term in word_tokenize(text, engine='attacut') if isthai(term)]

def preprocessText(review: pd.DataFrame):
    review['review'] = review['review'].map(lambda x : isGoogleTranslateAndNaN(str(x)))
    review.dropna(inplace=True)
    review = filterPlaceFromReviewNumber(review, numberReview=500)

    review['review'] = review['review'].map(lambda x : cleanText(x))

    return review

In [157]:
# Manage data
reviewClean = preprocessText(review=review)
print(f"Place left {len(reviewClean['placeName'].value_counts())} unit")
reviewClean

Place left 56 unit


,placeName,review
0,เซ็นทรัลเวิลด์,ห้างใหญ่มากอย่างที่กล่าวขานก็คือเดินจนหลงมากี่...
1,เซ็นทรัลเวิลด์,ศูนย์การค้า️เป็นศูนย์การค้าที่มีชื่อเดิมเรียกว...
2,เซ็นทรัลเวิลด์,ช่วงเทศกาลเที่ยวกันอย่าประมาทหน้ากากอนามัยถอดย...
3,เซ็นทรัลเวิลด์,เดินทางง่ายสะดวกไม่ว่าจะเป็นรถส่วนตัวแท็กซี่รถ...
4,เซ็นทรัลเวิลด์,สนุกกับช่วงเวลาแห่งการเฉลิมฉลองกับครอบครัวและค...
...,...,...
29495,ฮาราจูกุ ไทยแลนด์,สถานที่สวยงามเหมาะกับการนั่งรับปรัมานอาหารถ่าย...
29496,ฮาราจูกุ ไทยแลนด์,น่าจะโต๊ะนั่งทายมากว่านี้ซื้อของมาทานแล้วหาที่...
29497,ฮาราจูกุ ไทยแลนด์,สถานที่สวยงามมากๆเหมือนประเทศญี่ปุ่นที่ในเมือง...
29498,ฮาราจูกุ ไทยแลนด์,อาหารญี่ปุ่นชื่อร้านเรนอร่อยมาก


In [158]:
stop_words = [t.encode('utf-8') for t in list(thai_stopwords())]

tfidf_vector = TfidfVectorizer(
    tokenizer=text_tokenizer,
    ngram_range=(2,3),
    stop_words=stop_words,
    max_features=5000
)

In [159]:
X = tfidf_vector.fit_transform(reviewClean['review'])

In [160]:
term_tfidf = pd.DataFrame(
    X.toarray(),
    columns=tfidf_vector.get_feature_names(),
    index=reviewClean.index
)

c:\Users\Pongs\miniconda3\envs\TripRecommendation\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [161]:
term_tfidf

,กระเป๋า รองเท้า,กราบ ขอ,กราบ ขอ พร,กราบ พระ,กราบ สักการะ,กราบไหว้ ขอ,กราบไหว้ ขอ พร,กรุง ที่,กรุง รัตนโกสินทร์,กรุงเทพ ที่,...,ๆ โดย,ๆ ใน,ๆ ให้,ๆ ให้ เลือก,ๆ ได้,ๆ ไป,ๆ ไม่,ๆ ไม่ มี,ๆ ๆ,ๆ ๆ ๆ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.093287,0.121447,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
29496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
29497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
29498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


In [162]:
term_tfidf.sum().sort_values(ascending=False).head(20)

จอด รถ           456.446878
ที่ จอด          394.099670
ที่ จอด รถ       347.939755
เยอะ มาก         321.298622
มาก ๆ            313.320239
ดี มาก           311.821529
ของ กิน          255.878532
บรรยากาศ ดี      247.248029
ร้าน อาหาร       232.560832
ที่ นี่          227.404915
ออก กำลัง        212.841757
ร้าน ค้า         203.610187
ถ่าย รูป         200.563570
ให้ เลือก        199.297157
กำลัง กาย        197.378491
ออก กำลัง กาย    196.656670
มี ร้าน          194.787308
สวย มาก          186.567457
ดอก ไม้          185.352545
คน เยอะ          178.323002
dtype: float64

In [163]:
# Combination every review to 1 vector each place

numberPlace = len(reviewClean['placeName'].value_counts())
idx = 0
tfidfList = []

while idx < numberPlace:
    start = numberReview * idx
    end = start + numberReview

    i_tfidf = term_tfidf.iloc[start:end].sum() / numberReview

    tfidfList.append(i_tfidf.to_numpy())
    idx+=1

In [164]:
placeName = reviewClean['placeName'].unique()
tfidf_vector_dict = {}

for idx, place in enumerate(placeName):
     tfidf_vector_dict[place] = tfidfList[idx]

In [165]:
detail = pd.read_csv('../data/file/detailPlace.csv')
detail['tfidf'] = np.NaN
detail['tfidf'] = detail['tfidf'].astype(object)
detail.head(5)

,no,name,detail,business_status,address,geometry,phone_number,url,type,period,tfidf
0,1,เซ็นทรัลเวิลด์,"{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,"4, 5 ถนน ราชดำริ แขวงปทุมวัน เขตปทุมวัน กรุงเท...","(13.7460002, 100.5399162)",+66 2 640 7000,https://maps.google.com/?cid=14171986354817230340,"shopping_mall,point_of_interest,establishment","{'monday': '10:00–22:00', 'tuesday': '10:00–22...",NaN
1,2,สยามพารากอน,"{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,991 ถ. พระรามที่ ๑ แขวงปทุมวัน เขตปทุมวัน กรุง...,"(13.7462411, 100.5347402)",+66 2 690 1000,https://maps.google.com/?cid=11476178565854079331,"shopping_mall,department_store,store,point_of_...","{'monday': '10:00–22:00', 'tuesday': '10:00–22...",NaN
2,3,พระบรมมหาราชวัง,"{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,ถนน หน้าพระลาน แขวงพระบรมมหาราชวัง เขตพระนคร ก...,"(13.7498558, 100.4915765)",NaN,https://maps.google.com/?cid=3056950040631135297,"tourist_attraction,park,point_of_interest,esta...","{'monday': '8:30–15:30', 'tuesday': '8:30–15:3...",NaN
3,4,วัดพระศรีรัตนศาสดาราม (วัดพระแก้ว),"{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,QF2V+M34 ถนน หน้าพระลาน แขวงพระบรมมหาราชวัง เข...,"(13.7516435, 100.4927041)",NaN,https://maps.google.com/?cid=3764096327502462600,"tourist_attraction,place_of_worship,point_of_i...","{'monday': '8:30–15:30', 'tuesday': '8:30–15:3...",NaN
4,5,วัดอรุณราชวรารามราชวรมหาวิหาร (วัดแจ้ง),"{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,158 ถนน วังเดิม แขวงวัดอรุณ เขตบางกอกใหญ่ กรุง...,"(13.7442236, 100.488571)",+66 2 891 2185,https://maps.google.com/?cid=9727099538087655581,"place_of_worship,point_of_interest,establishment","{'monday': '8:00–18:00', 'tuesday': '8:00–18:0...",NaN


In [166]:
for idx in range(len(tfidf_vector_dict)):
    try:
        detail_place = detail.iloc[idx, 1]
        value = tfidf_vector_dict[detail_place]
        detail.iloc[idx, -1] = value.reshape(-1,1)
    except:
        continue

In [167]:
detail.dropna(inplace=True)
detail['no'] = [i+1 for i in range(len(detail))]
detail.to_csv('detailTFIDF.csv', index=False)

### TFIDF score & Recommend from term score

In [3]:
detail = pd.read_csv("detailTFIDF.csv")
detail.head(3)

,no,name,type2,detail,business_status,address,geometry,phone_number,url,type,period,tfidf
0,1,เซ็นทรัลเวิลด์,shopping,"{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,"4, 5 ถนน ราชดำริ แขวงปทุมวัน เขตปทุมวัน กรุงเท...","(13.7460002, 100.5399162)",+66 2 640 7000,https://maps.google.com/?cid=14171986354817230340,"shopping_mall,point_of_interest,establishment","{'monday': '10:00–22:00', 'tuesday': '10:00–22...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.000663240768079129..."
1,2,สยามพารากอน,shopping,"{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,991 ถ. พระรามที่ ๑ แขวงปทุมวัน เขตปทุมวัน กรุง...,"(13.7462411, 100.5347402)",+66 2 690 1000,https://maps.google.com/?cid=11476178565854079331,"shopping_mall,department_store,store,point_of_...","{'monday': '10:00–22:00', 'tuesday': '10:00–22...","[0.0002602116477231372, 0.0, 0.0, 0.0, 0.0, 0...."
2,3,วัดอรุณราชวรารามราชวรมหาวิหาร (วัดแจ้ง),"history,special","{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,158 ถนน วังเดิม แขวงวัดอรุณ เขตบางกอกใหญ่ กรุง...,"(13.7442236, 100.488571)",+66 2 891 2185,https://maps.google.com/?cid=9727099538087655581,"place_of_worship,point_of_interest,establishment","{'monday': '8:00–18:00', 'tuesday': '8:00–18:0...","[0.0, 0.004660260842552008, 0.0039156033393298..."


In [4]:
print(detail['tfidf'].to_numpy().shape)
print(np.array([np.fromstring(detail['tfidf'].to_numpy()[0][1:-1], sep=',')]).shape)

(45,)
(1, 5000)


In [5]:
# matrix of tfidf score
def get_termVector(detail: pd.DataFrame):
    term_vector = np.array([np.fromstring(detail['tfidf'].to_numpy()[0][1:-1], sep=',')])

    for i in range(len(detail) - 1):
        term_vector_i = np.array([np.fromstring(detail['tfidf'].to_numpy()[i+1][1:-1], sep=',')])
        term_vector = np.append(term_vector, term_vector_i, axis=0)
    
    return term_vector

In [6]:
term_vector = get_termVector(detail)
term_vector.shape

(45, 5000)

In [7]:
term_score = cosine_similarity(term_vector)
term_score.shape

(45, 45)

In [8]:
def recommend(place):
    index = detail[detail['name'] == place]['no'].values[0] - 1
    distances = sorted(list(enumerate(term_score[index].reshape(-1))), reverse=True, key=lambda x:x[1])
    for i in distances[1:11]:
        print(detail.iloc[i[0]]['name'])

In [10]:
recommend('วัดอรุณราชวรารามราชวรมหาวิหาร (วัดแจ้ง)')

วัดสุทัศนเทพวรารามราชวรมหาวิหาร
วัดนาคปรก
วัดสระเกศราชวรมหาวิหาร (ภูเขาทอง)
วัดไตรมิตรวิทยารามวรวิหาร
วัดประยุรวงศาวาสวรวิหาร
วัดธรรมมงคลเถาบุญนนทวิหาร(หลวงพ่อวิริยังค์)
เอเชียทีค เดอะ ริเวอร์ฟรอนท์
วัดปทุมวนาราม ราชวรวิหาร
พิพิธภัณฑ์ช้างเอราวัณ
วัดมังกรกมลาวาส (วัดเล่งเน่ยยี่)


## Full Recommendation system
- [x] TermScore
- [] RatingScore
- [] ReviewScore
- [] TypeScore

In [28]:
detail = pd.read_csv("detailTFIDF.csv")
detail.head(3)

,no,name,type2,detail,business_status,address,geometry,phone_number,url,type,period,tfidf
0,1,เซ็นทรัลเวิลด์,shopping,"{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,"4, 5 ถนน ราชดำริ แขวงปทุมวัน เขตปทุมวัน กรุงเท...","(13.7460002, 100.5399162)",+66 2 640 7000,https://maps.google.com/?cid=14171986354817230340,"shopping_mall,point_of_interest,establishment","{'monday': '10:00–22:00', 'tuesday': '10:00–22...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.000663240768079129..."
1,2,สยามพารากอน,shopping,"{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,991 ถ. พระรามที่ ๑ แขวงปทุมวัน เขตปทุมวัน กรุง...,"(13.7462411, 100.5347402)",+66 2 690 1000,https://maps.google.com/?cid=11476178565854079331,"shopping_mall,department_store,store,point_of_...","{'monday': '10:00–22:00', 'tuesday': '10:00–22...","[0.0002602116477231372, 0.0, 0.0, 0.0, 0.0, 0...."
2,3,วัดอรุณราชวรารามราชวรมหาวิหาร (วัดแจ้ง),"history,special","{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,158 ถนน วังเดิม แขวงวัดอรุณ เขตบางกอกใหญ่ กรุง...,"(13.7442236, 100.488571)",+66 2 891 2185,https://maps.google.com/?cid=9727099538087655581,"place_of_worship,point_of_interest,establishment","{'monday': '8:00–18:00', 'tuesday': '8:00–18:0...","[0.0, 0.004660260842552008, 0.0039156033393298..."


In [29]:
# Type from Place API (Too much type)

type_set = set()

for idx, type_i in enumerate(detail['type']):
    for t in type_i.split(','):
        type_set.add(t)

type_set, len(type_set)

({'amusement_park',
  'aquarium',
  'art_gallery',
  'bar',
  'department_store',
  'establishment',
  'museum',
  'park',
  'place_of_worship',
  'point_of_interest',
  'shopping_mall',
  'store',
  'tourist_attraction',
  'zoo'},
 14)

In [30]:
webDetail = pd.read_csv('../data/file/finalPlace.csv')
webDetail['use'] = np.NaN

In [31]:
placeName = detail['name']
len(placeName)

45

In [32]:
for idx, place in enumerate(webDetail['placeName']):
    for p in placeName:
        if place==p:
            webDetail.loc[idx, "use"] = 1

webDetail.dropna(inplace=True)
webDetail = webDetail.reset_index(drop=True)

In [35]:
detail.insert(2, "placeID", webDetail['placeId'])

True

In [16]:
# Type from Place API (Too much type)

type_set = set()

for idx, type_i in enumerate(detail['type2']):
    for t in type_i.split(','):
        type_set.add(t)

type_list = sorted(list(type_set))
type_list

['art', 'history', 'nature', 'shopping', 'special']

In [21]:
# Vector ['art', 'history', 'nature', 'shopping', 'special']
type_vector = np.zeros((len(detail), len(type_list)))

for idx in range(len(detail)):
    type_vector_i = np.zeros(5)
    for t in detail['type2'][idx].split(','):
        for i, t_i in enumerate(type_list):
            if t==t_i:
                type_vector_i[i]=1
    
    type_vector[idx] = type_vector_i

In [22]:
detail['type2'][3]

'history,nature'

In [23]:
type_vector[3]

array([0., 1., 1., 0., 0.])

In [24]:
type_score = cosine_similarity(type_vector)

type_score

array([[1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [25]:
def get_random_index(trip_level):
    # Find index of each type from type vector (Adding more History before TFIDF)
    idx_type_vector = {
        "art":[],
        "history":[],
        "nature":[],
        "shopping":[],
        "special":[]
    }

    for idx, type_vector_i in enumerate(type_vector):
        for i, x in enumerate(type_vector_i):
            if x==1:
                if i==0:
                    idx_type_vector["art"].append(idx)
                if i==1:
                    idx_type_vector["history"].append(idx)
                if i==2:
                    idx_type_vector["nature"].append(idx)
                if i==3:
                    idx_type_vector["shopping"].append(idx)
                if i==4:
                    idx_type_vector["special"].append(idx)

    # Random choices place index follow Trip level
    all_random_index = set()
    # Number of place art is 4
    try:
        random_index_art = random.sample(idx_type_vector['art'], k=trip_level[0])
    except:
        random_index_art = random.sample(idx_type_vector['art'], k=trip_level[0]-1)
    random_index_history = random.sample(idx_type_vector['history'], k=trip_level[1])
    random_index_nature = random.sample(idx_type_vector['nature'], k=trip_level[2])
    random_index_shopping = random.sample(idx_type_vector['shopping'], k=trip_level[3])

    for i in random_index_art: all_random_index.add(i)
    for i in random_index_history: all_random_index.add(i)
    for i in random_index_nature: all_random_index.add(i)
    for i in random_index_shopping: all_random_index.add(i)

    all_random_index = list(all_random_index)
    
    return all_random_index


def get_term_score(all_random_index):
    # Term vector
    # Select vector from upper index
    trip_term_vector = np.zeros(term_vector[0].shape)

    for idx_vector in all_random_index:
        trip_term_vector += term_vector[idx_vector]

    trip_term_vector = trip_term_vector / len(all_random_index)
    trip_term_score = cosine_similarity(np.append(term_vector, trip_term_vector.reshape(1,-1), axis=0))
    return trip_term_score

def get_type_score(all_random_index):
    # Type vector
    trip_type_vector = np.zeros(type_vector[0].shape)

    for idx_vector in all_random_index:
        trip_type_vector += type_vector[idx_vector]

    trip_type_vector = trip_type_vector / len(all_random_index)
    trip_type_score = cosine_similarity(np.append(type_vector, trip_type_vector.reshape(1,-1), axis=0))
    return trip_type_score

In [26]:
# Trip vector level 1-5 ['art', 'history', 'nature', 'special', 'shopping']
# This input emphasize the history place
# select level unit (mulitpy 1)
trip_level = [1,1,1,5] # Level ['art', 'history', 'nature', 'shopping']

random_index = get_random_index(trip_level)
trip_term_score = get_term_score(random_index)
trip_type_score = get_type_score(random_index)

# Rating score
rating_score = webDetail['rating'].to_numpy()
rating_score = np.append(rating_score, 5)

relevance_score = trip_term_score[-1]*0.5 + trip_type_score[-1]*0.3 + rating_score*0.2

# Recommendation
distances = sorted(list(enumerate(relevance_score.reshape(-1))), reverse=True, key=lambda x:x[1])
for i in distances[1:11]:
    # print(i[1])
    print(detail.iloc[i[0]]['name'])

ตลาดนัดรถไฟ ศรีนครินทร์
สยามพารากอน
เซ็นทรัลเวิลด์
เซ็นทรัล พระราม 2
เซ็นทรัล ลาดพร้าว
เซ็นทรัล พระราม 9
ตลาดอินดี้ ดาวคะนอง
ตลาดน้ำคลองลัดมะยม
ตลาดนัดเลียบด่วน รามอินทรา
ตลาดนกฮูก
